In [1]:
import os
import pandas as pd
import geopandas as gpd

import rasterio

import pystac_client
import planetary_computer as pc

import lidar_sampling_functions as lsf

## Downloading data through California Forest Observatory api

In [2]:
#pip install cfo

In [3]:
# import cfo
# #import gdal

In [4]:
# forest_api = cfo.api()
# forest_api.authenticate()

In [5]:
# forest_api.search(geography='California', metric='CanopyHeight')
# # msai4earth-esa/shapefiles_n_rasters/SB_geometry/SB_only.shp

In [6]:
# itemid = forest_api.search(geography='SantaBarbaraCounty', metric='CanopyHeight')[0]
# itemid

In [7]:
# lidar_file = os.path.join(os.getcwd(), 'santabarbara_lidar_2020.tif')

In [8]:
# forest_api.download(itemid, lidar_file)

## Open lidar and create min,max,avg,diff lidar rasters

In [9]:
lidar_year = 2020

In [10]:
fp = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)+'.tif')
lidar_rast_r = rasterio.open(fp)

In [11]:
lsf.save_min_max_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)),
                              aoi = 'SantaBarbaraCounty',
                              year = 2020)

lsf.save_avg_rasters(rast_reader = lidar_rast_r, 
                              folder_path = os.path.join(os.getcwd(),'SantaBarbaraCounty_lidar_'+str(lidar_year)),
                              aoi = 'SantaBarbaraCounty',
                              year = 2020)

## Open points from given year and aoi

In [12]:
aoi = 'campus_lagoon'
aoi_year = 2020

pts_fp = os.path.join('/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/',
                     aoi+'_points',
                     aoi+'_points_'+str(aoi_year)+'.csv')

## Obtain CRS from itemid and create geodataframe from points

In [13]:
itemid = pd.read_csv(pts_fp).naip_id[0]
itemid

'ca_m_3411934_sw_11_060_20200521'

In [14]:
# # accesing Azure storage using pystac client
# catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

# # search for naip scene where the pts were sampled from
# search = catalog.search(
#     collections=["naip"],
#     ids = itemid
# )
# item = list(search.get_items())[0]
# epsg_code = item.properties['proj:epsg']

In [15]:
pts = lsf.geodataframe_from_csv(pts_fp, lsf.crs_from_itemid(itemid))
pts.head(3)

NameError: name 'pystac_client' is not defined

## Sample LIDAR

In [ ]:
pts_xy = lsf.pts_for_lidar_sampling(pts, lidar_rast_r.crs)

In [ ]:
lidar_samples = lsf.sample_raster(pts_xy, lidar_rast_r)

In [ ]:
lidar_fps = []
for tag in ['maxs_', 'mins_', 'avgs_']:
    lidar_fps.append(os.path.join(os.getcwd(),
                                 'SantaBarbaraCounty_lidar_'+str(lidar_year),
                                 'SantaBarbaraCounty_lidar_'+tag+ str(lidar_year)+'.tif'))

In [ ]:
#fp = os.path.join(os.getcwd(),'lidar_maxs.tif')
maxs_rast_r = rasterio.open(lidar_fps[0])
max_samples = lsf.sample_raster(pts_xy, maxs_rast_r)

In [ ]:
mins_rast_r = rasterio.open(lidar_fps[1])
min_samples = lsf.sample_raster(pts_xy, mins_rast_r)

In [ ]:
#fp = os.path.join(os.getcwd(),'lidar_avgs.tif')
avg_rast_r = rasterio.open(lidar_fps[2])
avg_samples = lsf.sample_raster(pts_xy, avg_rast_r)

In [ ]:
pts['lidar'] = lidar_samples
pts['max_lidar']= max_samples
pts['min_lidar'] = min_samples
pts['min_max_diff'] = pts.max_lidar - pts.min_lidar
pts['avg_lidar'] = avg_samples


In [ ]:
pts.head(3)

In [ ]:
# ptslidar_fp = os.path.join('/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/',
#                      aoi+'_points',
#                      aoi+'_points_lidar'+str(aoi_year)+'.csv')
# pts.to_csv(ptslidar_fp)

In [ ]:
for fp in lidar_fps:
    os.remove(fp)